In [ ]:
import pandas as pd
import numpy as np

csv_df = pd.read_csv('H28.csv',encoding="utf8") 
len(csv_df)
csv_df.info()
csv_df.head()

csv_df2 = pd.read_csv('H29.csv',encoding="utf8") 
len(csv_df2)
csv_df2.info()
csv_df2.head()

csv_df3 = pd.read_csv('H30.csv',encoding="utf8") 
len(csv_df3)
csv_df3.info()
csv_df3.head()

csv_df4 = pd.read_csv('R1.csv',encoding="utf8") 
len(csv_df4)
csv_df4.info()
csv_df4.head()

csv_df5 = pd.read_csv('R2.csv',encoding="utf8") 
len(csv_df5)
csv_df5.info()
csv_df5.head()

In [ ]:
#関数の定義
#データのスケール変換の関数（０〜１のスケールに変換）
def scaler(array):
    
    M = max(array)
    m = min(array)
    if M != m:
        scaled_array = (array - m*np.ones(shape = array.shape))/(M - m)
        def f(x):
            return (x - m)/(M - m)
    else:
        scaled_array = array - m*np.ones(shape = array.shape)
        def f(x):   
            return x - m
#scaled_arrayは変換後のデータ,fは個別に値が与えられた時にscaled_arrayのデータと同様に変換するための関数            
    return scaled_array, f

#データの前処理の関数
def preprocessing(x):
    xl = []
    prpl = []
    for i in range(len(x.T)):
        l = scaler(x.T[i])[0]
        prpl.append(scaler(x.T[i])[1])
        xl.append(l)
    
    return np.array(xl).T, prpl #np.array(xl).Tは前処理後のデータ,prplは各項目のスケールに合わせるための関数を格納した配列

#最も近い値に値を変換する関数
#モデルから出力された値を最も近い実際の地目コードに変換する
def nearvalue(x, l):
    d = []
    for i in range(len(l)):
        d.append(abs(x-l[i]))
        
    return l[d.index(min(d))]

#決定係数を計算する関数
#決定係数を求めるメソッドがないモデルなどの検証に使う
def r_2(v, w):
    
    s1 = np.sum((v-w)**2)
    s2 = np.sum((v-np.mean(v))**2)
    
    return 1-(s1/s2)

#正解率を計算する関数
#地目コードとの一致率を計算するメソッドがないモデルなどの検証に使う
def accuracy(v,w):
    u = v-w
    a = 0
    for d in u:
        if d != 0:
            a+=1
            
    return 1-(a/len(u))

In [ ]:
#データの前処理のプログラム
import numpy as np
import time
import sklearn
ev = csv_df.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev2 = csv_df2.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev3 = csv_df3.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev4 = csv_df4.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev5 = csv_df5.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])

x_H28 = np.array(ev.values)
y_H28 = np.array(csv_df['現況地目コード'].values)
x_H29 = np.array(ev2.values)
y_H29 = np.array(csv_df2['現況地目コード'].values)
x_H30 = np.array(ev3.values)
y_H30 = np.array(csv_df3['現況地目コード'].values)
x_R1 = np.array(ev4.values)
y_R1 = np.array(csv_df4['現況地目コード'].values)
x_R2 = np.array(ev5.values)
y_R2 = np.array(csv_df5['現況地目コード'].values)

x =np.concatenate([x_H28, x_H29])
x =np.concatenate([x, x_H30])
x =np.concatenate([x, x_R1])
x =np.concatenate([x, x_R2])
y =np.concatenate([y_H28, y_H29])
y =np.concatenate([y, y_H30])
y =np.concatenate([y, y_R1])
y =np.concatenate([y, y_R2])
y_class = list(set(y))

prpx = preprocessing(x)[1]
x = preprocessing(x)[0]
x_H28 = x[0:len(x_H28)]
x_H29 = x[len(x_H28):len(x_H28)+len(x_H29)]
x_H30 = x[len(x_H28)+len(x_H29):len(x_H28)+len(x_H29)+len(x_H30)]
x_R1 = x[len(x_H28)+len(x_H29)+len(x_H30):len(x_H28)+len(x_H29)+len(x_H30)+len(x_R1)]
x_R2 = x[len(x_H28)+len(x_H29)+len(x_H30)+len(x_R1):]

y_H28 = y_H28[np.argsort(x_H28[:,0])]
y_H29 = y_H29[np.argsort(x_H29[:,0])]
y_H30 = y_H30[np.argsort(x_H30[:,0])]
y_R1 = y_R1[np.argsort(x_R1[:,0])]
y_R2 = y_R2[np.argsort(x_R2[:,0])]

x_H28 = x_H28[np.argsort(x_H28[:,0])]
x_H29 = x_H29[np.argsort(x_H29[:,0])]
x_H30 = x_H30[np.argsort(x_H30[:,0])]
x_R1 = x_R1[np.argsort(x_R1[:,0])]
x_R2 = x_R2[np.argsort(x_R2[:,0])]

In [ ]:
#H28を学習しH29をテスト
x_train = x_H28
y_train = y_H29[:len(x_H28)]
x_test = x_H29
y_test = y_H30[:len(x_H29)]

from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=10)

start = time.time()

dtr.fit(x_train, y_train)

y_tr_predict = np.array([nearvalue(i, y_class) for i in dtr.predict(x_train)])
y_predict = np.array([nearvalue(i, y_class) for i in dtr.predict(x_test)])

from sklearn.metrics import mean_squared_error
import sklearn
r2_1 = r_2(y_test, y_predict)
accuracy1 = accuracy(y_predict,y_test)
print(f"決定係数{r2_1}")
print(f"正解率{accuracy1}")

accuracy1 = dtr.score(x_test, y_test)
print(f"決定係数{accuracy1}")



#print("classification report")
#print(sklearn.metrics.classification_report(y_test, predicted))

t2 =time.time() - start
print("かかった時間")
print(t2)

In [ ]:
#H28とH29を学習しH30をテスト
x_train2 = np.concatenate([x_train, x_test])
y_train2 = np.concatenate([y_train, y_test])
x_test2 = x_H30
y_test2 = y_R1[:len(x_H30)]


from sklearn import neural_network
dtr2 = DecisionTreeRegressor(max_depth=10)

start = time.time()

dtr2.fit(x_train2, y_train2)

y_tr_predict2 = np.array([nearvalue(i, y_class) for i in dtr2.predict(x_train2)])
y_predict2 = np.array([nearvalue(i, y_class) for i in dtr2.predict(x_test2)])

from sklearn.metrics import mean_squared_error
import sklearn
r2_2 = r_2(y_test2, y_predict2)
accuracy2 = accuracy(y_test2,y_predict2)
print(f"決定係数{r2_2}")
print(f"正解率{accuracy2}")

accuracy2 = dtr2.score(x_test2, y_test2)
print(f"決定係数{accuracy2}")

#predicted2 = clf2.predict(x_test2)

#print("classification report")
#print(sklearn.metrics.classification_report(y_test2, predicted2))

t2 =time.time() - start
print("かかった時間")
print(t2)

In [ ]:
#H28~H30を学習しR1をテスト
x_train3 = np.concatenate([x_train2, x_test2])
y_train3 = np.concatenate([y_train2, y_test2])
x_test3 = x_R1
y_test3 = y_R2[:len(x_R1)]



from sklearn import neural_network
dtr3 = DecisionTreeRegressor(max_depth=10)

start = time.time()

dtr3.fit(x_train3, y_train3)

y_tr_predict3 = np.array([nearvalue(i, y_class) for i in dtr3.predict(x_train3)])
y_predict3 = np.array([nearvalue(i, y_class) for i in dtr3.predict(x_test3)])

from sklearn.metrics import mean_squared_error
import sklearn
r2_3 = r_2(y_test3, y_predict3)
accuracy3 = accuracy(y_test3,y_predict3)
print(f"決定係数{r2_3}")
print(f"正解率{accuracy3}")

accuracy3 = dtr3.score(x_test3, y_test3)
print(f"決定係数{accuracy3}")

#predicted3 = clf3.predict(x_test3)

#print("classification report")
#print(sklearn.metrics.classification_report(y_test3, predicted3))

t2 =time.time() - start
print("かかった時間")
print(t2)